# Хранилища

Структура занятия:

1) sql

2) mongodb

3) redis

База данных – совокупность данных, отражающая состояние объектов и их отношений в заданной предметной области. Имеет название и разные свойства. Чаще всего БД выражены разного рода таблицами и списками. 

Управление базами данных осуществляется через СУБД.

Базы данных бывают:

    - Фотографическими. В них информация представлена фактами об элементах предметной области в формате «параметр-значение».
    - Документальными. Информация преподносится полнотекстовыми документами.
    - Мультимедийными. Здесь допускается графика, видео и аудиоконтент.

Также есть классификация соответствующих хранилищ по типу используемой модели данных. Все БД условно делятся на:

    - Реляционные. В них информация преподносится в виде таблиц, а также разного рода связей между ними. К таковым относят: SQL Server, MySQL, PostgreSQL.
        - Обычные (вертикальные) пример - PostgreSQL, MySQL
        - Аналитические (колоночные)  пример - Vertica, ClickHouse
    - Нереляционные. Данные тут представлены структурами, которые отличаются от таблиц. Пример – MongoDB. Сведения могут выступать в виде JSON-подобных элементов, сетевых структур или иерархий.
        - Документноориентированные, пример - MongoDB, CockroachDB
        - Типа ключ-значение, пример - Redis
        - Файловые, пример - S3
    - Очереди сообщений. Служат для асинхронной передачи данных по модели продьюсер-консьюмер (паблишер-сабскрайбер)
    
    
Отличаются БД и по топологии хранения:

    - Локальные. Они размещаются и хранятся в пределах одного устройства.
    - Распределенные. Базы информации будут размещаться непосредственно на нескольких машинах.

## Реляционные БД. SQL

Реляционная база данных – это набор данных с предопределенными связями между ними. Эти данные организованны в виде набора таблиц, состоящих из столбцов и строк. В таблицах хранится информация об объектах, представленных в базе данных. В каждом столбце таблицы хранится определенный тип данных, в каждой ячейке – значение атрибута. Каждая стока таблицы представляет собой набор связанных значений, относящихся к одному объекту или сущности. Каждая строка в таблице может быть помечена уникальным идентификатором, называемым первичным ключом, а строки из нескольких таблиц могут быть связаны с помощью внешних ключей. 

Отношения (таблицы) обладают свойcтвом [Нормальной формы](https://ru.wikipedia.org/wiki/%D0%9D%D0%BE%D1%80%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D1%84%D0%BE%D1%80%D0%BC%D0%B0)

Рассмотрим реляционные БД на примере сервиса постов (типа Твиттера). Структура БД будет включать 4 таблицы: 
1) `users`: id, name, age
2) `posts`: id, title, text, user_id (1 user: N post)
3) `likes`: id, user_id (1 user: N like), post_id (1 post: N like)
4) `comments`: id, text, user_id (1 user: N comment), post_id (1 post: N comment)

https://www.db-fiddle.com/f/gfPDhUXHQTTPet6hpktoZq/0

Для PostgreSQL в стандартной библиотеке Python модуля нет. Воспользуемся `psycopg2`

!Важно. Для того чтобы подключиться к БД, необхлдимо развернуть postgresql на локальной машине. Легче всего сделать это при помощи docker: достаточно просто запустить [соответствующий контейнет](https://hub.docker.com/_/postgres), при этом не забудьте пробросить порт postgresql 5432 на локальный (в моём случае также 5432)

In [1]:
pip install psycopg2-binary


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Проверим подключение к PostgreSQL 

In [146]:
import psycopg2
try:
    # пытаемся подключиться к базе данных
    conn = psycopg2.connect('postgresql://postgres@localhost:5432/postgres')
    cur = conn.cursor()
except:
    # в случае сбоя подключения будет выведено сообщение  в STDOUT
    print('Can not establish connection to database')
finally:
    cur.close()
    conn.close()


Объект соединения `connect` имеет следующие основные атрибуты:
- `__enter__(...)` и `__exit__(...)`
- property `autocommit` - включает коммит (применение изменений) на каждую операцию
- `commit()` - применяет все (с момента последнего коммита) изменения в БД 
- `rollback()` - отменяет все (с момента последнего коммита) изменения в БД
- `cancel()` - отменяет текущую операцию
- `cursor()` - возвращает новый курсор

`commit()` и `rollback()` - элементы работы с транзакциями. [Транза́кция](https://ru.wikipedia.org/wiki/%D0%A2%D1%80%D0%B0%D0%BD%D0%B7%D0%B0%D0%BA%D1%86%D0%B8%D1%8F_(%D0%B8%D0%BD%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B0)) — группа последовательных операций с базой данных, которая представляет собой логическую единицу работы с данными. Транзакция может быть выполнена либо целиком и успешно, соблюдая целостность данных и независимо от параллельно идущих других транзакций, либо не выполнена вообще, и тогда она не должна произвести никакого эффекта. Транзакции обрабатываются транзакционными системами, в процессе работы которых создаётся история транзакций. 

Объект курсора `сursor` имеет следующие основные атрибуты:
- `__enter__(...)` и `__exit__(...)`
- `__iter__(self, /)` и `__next__(self, /)`
- `close()` - закрывает курсор
- `execute(query, vars=None)` - исполняет SQL запрос
- `executemany(query, vars_list)` - исполняет SQL запрос на множестве входных аргументов
- `fetchone()` - возврашает 1 объект данных (1 строку) из запрошенных
- `fetchmany(size=self.arraysize)` - возбращает `size` объектов данных из запрошенных
- `fetchall()` - возвращает все запрошенные данные

Создадим таблицы, в соответствии с моделью описаной выше

Для того чтобы создавать таблицы в SQL используется слово `CREATE TABLE` (оегистр не исеет значения, но хорошим тоном является использование заглавных букв), `IF NOT EXISTS` - сообщает что таблица будет создана только если она ещё не существует, за скобками перечисляются поля (названия и типы колонок).

In [147]:
query = """
CREATE TABLE IF NOT EXISTS users (
  id SERIAL PRIMARY KEY,
  name TEXT NOT NULL, 
  age INTEGER
);
CREATE TABLE IF NOT EXISTS posts (
  id SERIAL PRIMARY KEY, 
  title TEXT NOT NULL, 
  text TEXT NOT NULL, 
  user_id INTEGER REFERENCES users(id)
);
CREATE TABLE IF NOT EXISTS comments (
  id SERIAL PRIMARY KEY, 
  text TEXT NOT NULL, 
  user_id INTEGER REFERENCES users (id), 
  post_id INTEGER REFERENCES posts (id)
);
CREATE TABLE IF NOT EXISTS likes (
  id SERIAL PRIMARY KEY, 
  user_id INTEGER REFERENCES users (id), 
  post_id integer REFERENCES posts (id)
);
"""
with psycopg2.connect('postgresql://postgres@localhost:5432/postgres') as conn:
    with conn.cursor() as curr:
        curr.execute(query)
    conn.commit()

Для наполнения таблиц данными, в SQL применяется слово `INSERT INTO`

Вставка данных. 1 способ: execute строки

In [148]:
query = """
INSERT INTO
  users (name, age)
VALUES
  ('Vanya', 25),
  ('Lesha', 32),
  ('Masha', 20),
  ('Dima', 16),
  ('Natasha', 21);
"""
with psycopg2.connect('postgresql://postgres@localhost:5432/postgres') as conn:
    conn.autocommit = True
    with conn.cursor() as curr:
        curr.execute(query)

Вставка данных. 2 способ: execute объединённых строк (кривой)

In [149]:
posts = [
  ('Много работы','Очень много работы', 1),
  ('Мало работы', 'Ищу работу', 1),
  ('Помогите', 'Помогите выбрать дом', 2),
  ('Новость', 'Я счастлив!', 1),
  ('Сад', 'Надо купить прострел', 3),
]
posts_records = ", ".join(["%s"] * len(posts))

insert_query = (
    f"INSERT INTO posts (title, text, user_id) VALUES {posts_records}"
)
with psycopg2.connect('postgresql://postgres@localhost:5432/postgres') as conn:
    conn.autocommit = True
    with conn.cursor() as curr:
        curr.execute(insert_query, posts)

Вставка данных. 3 способ: executemany

In [151]:
likes = [
  (1, 5),
  (2, 3),
  (1, 3),
  (2, 4),
  (1, 4),
  (2, 2),
  (3, 4),
]
insert_query = (
    f"INSERT INTO likes (user_id, post_id) VALUES (%s, %s)"
)
with psycopg2.connect('postgresql://postgres@localhost:5432/postgres') as conn:
    conn.autocommit = True
    with conn.cursor() as curr:
        curr.executemany(insert_query, likes)

В SQL данные извлекаются при помощи слова `SELECT`, при этом оперция выборки очень многообразна... отношения (таблицы) в процессе выборки могут быть объединины между собой (`JOIN`), данные отфильтрованы по значениям (`WHERE`) или дополнены (`OVER`), сгруппированы по какому либо правилу (`GROUP BY`), упорядочены (`ORDER BY`)

Для извлечения данных также необходимо выполнить запрос `execute`, и далее осуществить выборку

In [153]:
select_query = """
SELECT * FROM users AS u RIGHT JOIN posts AS p ON u.id = p.user_id 
WHERE u.age > 20 OR u.age < 15 ORDER BY u.age DESC
"""
# try: RIGHT JOIN -> LEFT JOIN; 

with psycopg2.connect('postgresql://postgres@localhost:5432/postgres') as conn:
    conn.autocommit = True
    with conn.cursor() as curr:
        curr.execute(select_query)
        result = curr.fetchall()
        
for i in result:
    print(i)

(2, 'Lesha', 32, 8, 'Помогите', 'Помогите выбрать дом', 2)
(2, 'Lesha', 32, 3, 'Помогите', 'Помогите выбрать дом', 2)
(1, 'Vanya', 25, 4, 'Новость', 'Я счастлив!', 1)
(1, 'Vanya', 25, 6, 'Много работы', 'Очень много работы', 1)
(1, 'Vanya', 25, 7, 'Мало работы', 'Ищу работу', 1)
(1, 'Vanya', 25, 1, 'Много работы', 'Очень много работы', 1)
(1, 'Vanya', 25, 9, 'Новость', 'Я счастлив!', 1)
(1, 'Vanya', 25, 2, 'Мало работы', 'Ищу работу', 1)


В приложениях запросы формируются на основании бизнес-логики в функциях или методах

In [154]:
def get_most_popular_posts(cur, count) -> list[tuple[int, int]]:   # (post_id, popularity)
    select_query = f"""
    SELECT p.id, count(l.id) as rank FROM posts AS p LEFT JOIN likes AS l ON p.id = l.post_id 
    GROUP BY p.id ORDER BY rank DESC LIMIT {count}
    """

    curr.execute(select_query)
    result = curr.fetchall()
    return result

Для обновления данных используется слово `UPDATE`

In [155]:
update_post_query = """
UPDATE posts
SET text = 'Купили прострел'
WHERE id = 5
"""
with psycopg2.connect('postgresql://postgres@localhost:5432/postgres') as conn:
    with conn.cursor() as curr:
        curr.execute(update_post_query)
    conn.rollback()

Для удаления данных используется слово `DELETE`

In [156]:
insert_comment_query = """
INSERT INTO comments (text, user_id, post_id) VALUES ('Такое себе', 2, 2)
"""
with psycopg2.connect('postgresql://postgres@localhost:5432/postgres') as conn:
    with conn.cursor() as curr:
        curr.execute(insert_comment_query)
    conn.commit()

In [157]:
delete_comment_query = """
DELETE FROM comments WHERE id = 1
"""
with psycopg2.connect('postgresql://postgres@localhost:5432/postgres') as conn:
    with conn.cursor() as curr:
        curr.execute(delete_comment_query)
    conn.commit()

SQL предоставляет очень широкие воpзможности, обозначим ещё некоторые из них:
- хранимые процедуры - своеобразные функции, выполняемые в БД
- триггеры - хранимые процедуры, выполняемые автоматически на операциях модификации данных
- возможность создания представлений 

Также в python существуют библиотеки для работы с объектрно-реляционными отображениями (ORM), которые решают задачу автоматизации работы с БД, скрывая чистый SQL. Например - sqlalchemy

## Документноориентированные БД. MongoDB

MongoDB - NoSQL база данных. Хранит данные в виде документов (json, html, text, ...)

В MongoDB все данные хранятся в json-подобном формате BSON: данные, как в json, представимы в виде объектов ключ-значение, однако BSON продоставляет несколько дополнительных типов, например date, uuid, binary. Максимальный размер 1-го документа - 16 МБ.

Документы хранятся в коллекциях, коллекции в базах данных. Каждая коллекция является отдельным файлом 

Далее рассмотрим особенности MongoDB:
- хорошая масштабируемость
- отношения между документами отсутствуют
- данные могут быть неструктурированными (иметь произвольный формат)
- мало экспертов высокого уровня (сложности администрирования больших кластеров)

!Важно. Для того чтобы подключиться к БД, необхлдимо развернуть mongodb на локальной машине. Легче всего сделать это при помощи docker: достаточно просто запустить соответствующий контейнет, при этом не забудьте пробросить порт mongodb 27017 на локальный (в моём случае также 27017)

Для работы с MongoDB потребуется установить библиотеку `pymongo`

In [158]:
pip install pymongo


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [159]:
import pymongo

mongo_uri = 'mongodb://localhost:27017'

client = pymongo.MongoClient(mongo_uri)

db = client['test-db']  # создаём БД (на самом деле будет создана только после записи 1-го документа)
coll = db['test-collection']  # создаём коллекцию (на самом деле будет создана только после записи 1-го документа)
coll.drop()

Для записи документов служат методы коллекции `insert_one` и `insert_many`

Запишем документ

In [160]:
from datetime import datetime
result = coll.insert_one({
    'brand': 'BMW', 
    'model': 'X6', 
    'contact': {'name': 'Vasya', 'age': 33}, 
    'TO': [2020, 2022, 2023],
    'date': datetime(2023, 5, 5, 8, 30, 12)
})

In [161]:
result.inserted_id

ObjectId('6452447ba19f30fc834833c9')

In [162]:
coll.insert_one({
    'brand': 'Audi', 
    'model': '6', 
    'contact': {'name': 'Vasya', 'age': 31}, 
    'date': datetime(2023, 5, 5, 8, 31, 12)
})

In [163]:
coll.insert_one({
    'brand': 'LADA', 
    'model': '10', 
    'contact': {'name': 'Dima', 'age': 66}, 
    'date': datetime(2023, 5, 5, 8, 34, 11),
    'history': [{'2010': 'broken'}, {'2021': 'repaired'}, {'2023': 'broken'}]
})

При записе каждому документу MongoDB добавляет новый ключ \_id - это уникальный ключ документа, рассчитываемый на основе timestamp-а на клиенте и некоторой случайной величины (при этом timestamp может быть восстановлен).

Получить документы можно при помощи методов 
- `find({search_query}[, {filter_query}])`
- `find_one({search_query}[, {filter_query}])`

эти функции вернут курсор (итератор с данными)

In [164]:
cur = coll.find({})
for i in cur:
    print(i)

{'_id': ObjectId('6452447ba19f30fc834833c9'), 'brand': 'BMW', 'model': 'X6', 'contact': {'name': 'Vasya', 'age': 33}, 'TO': [2020, 2022, 2023], 'date': datetime.datetime(2023, 5, 5, 8, 30, 12)}
{'_id': ObjectId('6452447ba19f30fc834833ca'), 'brand': 'Audi', 'model': '6', 'contact': {'name': 'Vasya', 'age': 31}, 'date': datetime.datetime(2023, 5, 5, 8, 31, 12)}
{'_id': ObjectId('6452447ca19f30fc834833cb'), 'brand': 'LADA', 'model': '10', 'contact': {'name': 'Dima', 'age': 66}, 'date': datetime.datetime(2023, 5, 5, 8, 34, 11), 'history': [{'2010': 'broken'}, {'2021': 'repaired'}, {'2023': 'broken'}]}


In [165]:
cur = coll.find({'contact.name': 'Dima'})
for i in cur:
    print(i)

{'_id': ObjectId('6452447ca19f30fc834833cb'), 'brand': 'LADA', 'model': '10', 'contact': {'name': 'Dima', 'age': 66}, 'date': datetime.datetime(2023, 5, 5, 8, 34, 11), 'history': [{'2010': 'broken'}, {'2021': 'repaired'}, {'2023': 'broken'}]}


In [166]:
cur = coll.find({'contact.name': 'Vasya'}, {'model': 1})
for i in cur:
    print(i)

{'_id': ObjectId('6452447ba19f30fc834833c9'), 'model': 'X6'}
{'_id': ObjectId('6452447ba19f30fc834833ca'), 'model': '6'}


Обновление документов

In [167]:
find_query = {'contact.name': 'Dima'}
update_query = {'contact.name': 'Dimon'}
coll.update_one(find_query, {'$set': update_query})

In [168]:
cur = coll.find({'contact.name': {'$ne': 'Vasya'}})  # все не Vasya
for i in cur:
    print(i)

{'_id': ObjectId('6452447ca19f30fc834833cb'), 'brand': 'LADA', 'model': '10', 'contact': {'name': 'Dimon', 'age': 66}, 'date': datetime.datetime(2023, 5, 5, 8, 34, 11), 'history': [{'2010': 'broken'}, {'2021': 'repaired'}, {'2023': 'broken'}]}


Удаление документов

In [169]:
query = {'contact.name': 'Dimon'}
coll.delete_one(query)

In [170]:
cur = coll.find({'contact.name': {'$ne': 'Vasya'}})  # все не Vasya
for i in cur:
    print(i)

Агрегации - механизм построения сложных запросов, позволяют в том числе, объединять документы из различных коллекций

In [171]:
cur = coll.aggregate([
    {'$match': {'contact.name': 'Vasya', 'contact.age': {'$gte': 32}}},
    {'$project': {'name': '$contact.name', 'brand': 1, 'model': 1}}
])
for i in cur:
    print(i)

{'_id': ObjectId('6452447ba19f30fc834833c9'), 'brand': 'BMW', 'model': 'X6', 'name': 'Vasya'}


## БД ключ-значение. Redis

Redis означает Remote Dictionary Service

База данных Redis содержит пары key: value и поддерживает такие команды как GET, SET и DEL. Ключи всегда строки. Значения - строки, списки, множества, хеши, словари. На ключи может быть установлен TTL (спустя время ttl ключ будет автоматически удалён)

Основные области применения:
- кеш
- персистентное хранилище
- распределённые мьютексы и счётчики
- очереди

`redis-py` — клиентская библиотека Python, позволяет общаться с сервером Redis напрямую через вызовы Python

In [172]:
!python -m pip install redis


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [179]:
import redis

r = redis.Redis()
pipe = r.pipeline()  # транзакции (pipeline) предоставляют в точности такой же интерфейс как и Redis()
pipe.set('k1', 'v1')
pipe.mset({'k2': 'v2', 'k3': 'v3'})
pipe.execute()

[True, True]

In [180]:
r.mset({'k2': 'v2', 'k3': 'v3'})

True

In [181]:
r.keys()  # вывести все ключи

[b'k3', b'k2', b'k1']

In [182]:
r.set('k4', 'v4', ex=1)  # expire, ключ пропадёт через 1 секунду
r.keys()

[b'k3', b'k4', b'k2', b'k1']

In [183]:
import time

time.sleep(1)
r.keys()

[b'k3', b'k2', b'k1']

In [184]:
r.flushdb()  # очистка

True

Редис также предоставляет механизм транзаций (pipeline-ов в терминах самого редиса).

Хранимые процедуры реализуются с помощью скриптов lua, которые однажды загружаются на сервер и далее выполняются на сервере редиса.

## Что ещё полезно знать о хранилищах

1) Шардирование. Сегментирование (англ. sharding) — подход, предполагающий разделение баз данных, отдельных её объектов или индексов поисковых систем на независимые сегменты, каждый из которых управляется отдельным экземпляром сервера базы данных, размещаемым, как правило, на отдельном вычислительном узле. 

2) Реплицирование. Репликация (англ. replication) — механизм синхронизации содержимого нескольких копий объекта (например, содержимого базы данных). Репликация — это процесс, под которым понимается копирование данных из одного источника на другой (или на множество других) и наоборот. 

3) Индексирование. Индекс (англ. index) — объект базы данных, создаваемый с целью повышения производительности поиска данных. Таблицы в базе данных могут иметь большое количество строк, которые хранятся в произвольном порядке, и их поиск по заданному критерию путём последовательного просмотра таблицы строка за строкой может занимать много времени. Индекс формируется из значений одного или нескольких столбцов таблицы и указателей на соответствующие строки таблицы и, таким образом, позволяет искать строки, удовлетворяющие критерию поиска. Ускорение работы с использованием индексов достигается в первую очередь за счёт того, что индекс имеет структуру, оптимизированную под поиск — например, сбалансированного дерева. 